### Python 3.6 - Dictionary Ordering

Python 3.6 sees a new implementation of the standard `dict` that preserves key ordering (based on the order in which new keys are added into or removed from the dictionary)

Although this is an implementation detail in 3.6, it is supposed to become official in Python 3.7. 

`https://mail.python.org/pipermail/python-dev/2017-December/151283.html`

So it should be safe to now assume dictionary keys will retain order , and using an `OrderedDict` might no longer be needed for certain cases. I'll come back to `OrderedDict` in a bit...

**Use for Python 3.6 and higher only!**

If you use that ordering feature with standard `dict` objects and then try to run your code in a prior version, things will break :-)



Also means that `keys()` (that iterate from left to right) preserves order as does `values()`. On the other hand `popitem()` will pop right-most element from dictionary. Sounds like stack behavior to me - but does not mean you should use it as such - I'm pretty sure that's not going to be as efficient as say using a list!.

Let's see some of this (just make sure you're running 3.6 or higher)

In [1]:
from sys import version_info

In [2]:
version_info

sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)

In [3]:
d = {'a': 1, 'b': 2}

In [4]:
d.keys(), d.values(), d.items()

(dict_keys(['a', 'b']), dict_values([1, 2]), dict_items([('a', 1), ('b', 2)]))

In [5]:
d['x'] = 3

In [6]:
d.keys(), d.values(), d.items()

(dict_keys(['a', 'b', 'x']),
 dict_values([1, 2, 3]),
 dict_items([('a', 1), ('b', 2), ('x', 3)]))

In [7]:
del d['b']

In [8]:
d.keys(), d.values(), d.items()

(dict_keys(['a', 'x']), dict_values([1, 3]), dict_items([('a', 1), ('x', 3)]))

In [9]:
d['b'] = 4

In [10]:
d.keys(), d.values(), d.items()

(dict_keys(['a', 'x', 'b']),
 dict_values([1, 3, 4]),
 dict_items([('a', 1), ('x', 3), ('b', 4)]))

What about replacing a value for an existing key?

In [11]:
d['x'] =100

In [12]:
d.keys(), d.values(), d.items()

(dict_keys(['a', 'x', 'b']),
 dict_values([1, 100, 4]),
 dict_items([('a', 1), ('x', 100), ('b', 4)]))

Order maintained...

And if we pop:

In [13]:
d.popitem()

('b', 4)

We popped the last item.

So now I'm left wondering how to pick a *random* item from the dictionary?
(Not that we could before - it might have looked random when we popped an item but it wasn't)

Sounds like another video :-)

**Important note:** Be careful of Jupyter notebooks - something I just realized - take a look:

In [14]:
d = {'x': 1, 'a': 2}

In [15]:
print(d)

{'x': 1, 'a': 2}


In [16]:
d

{'a': 2, 'x': 1}

Notice how just letting Jupyter display `d` changed the display order of the keys? (I'm guessing its doing something similar to `prettyprint` (or maybe even using it), which changes the displayed key order to be lexicographical.

What about using `update` to update one dict based on another (inserting missing keys, and updating common ones - but especially the insertion bit) - I'm guessing the order is retained with any insertions appearing at the end and ordered according to the dict being merged in. Let's see:

In [17]:
d1 = {'a': 1, 'b': 200}
d2 = {'a': 100, 'd':300, 'c':400}

In [18]:
d1.update(d2)

In [19]:
print(d1)

{'a': 100, 'b': 200, 'd': 300, 'c': 400}


Back to `OrderedDict` for a second:

`OrderedDict` has a few methods that afaik have no equivalent (yet) in standard dicts:
* `move_to_end(key, last=True)` - either move to front or back
* `popitem(last=True)` - that can pop either from front or back of dict
* supports reverse iteration using `reversed()`

Let's take each one of those one by one:

**move to end:**

In [20]:
d = {'a':1, 'b':2, 'c':3}
print('start:', d)
d['a'] = d.pop('a')
print('moved a to end:',d)

start: {'a': 1, 'b': 2, 'c': 3}
moved a to end: {'b': 2, 'c': 3, 'a': 1}


**move to front:**

That's one not so easy, the only option I can think of is to move all the keys around to re-order. Maybe something like this:

In [21]:
d = {'a':1, 'b':2, 'c':3, 'x':100, 'y':200}
print('start:', d)
d['c'] = d.pop('c')
print('moved c to end:', d)

for i in range(len(d)-1):
    key = next(iter(d.keys()))
    d[key] = d.pop(key)
print('moved c to front:', d)

start: {'a': 1, 'b': 2, 'c': 3, 'x': 100, 'y': 200}
moved c to end: {'a': 1, 'b': 2, 'x': 100, 'y': 200, 'c': 3}
moved c to front: {'c': 3, 'a': 1, 'b': 2, 'x': 100, 'y': 200}


**pop last item:**

That one's trivial - just use `popitem`:

In [22]:
d = {'a':1, 'b':2, 'c':3, 'x':100, 'y':200}
print('start:', d)
d.popitem()
print('pop last item:', d)

start: {'a': 1, 'b': 2, 'c': 3, 'x': 100, 'y': 200}
pop last item: {'a': 1, 'b': 2, 'c': 3, 'x': 100}


**pop first item:**

Not too difficult - we just need to identify the *first* key, and pop it.

In [23]:
d = {'a':1, 'b':2, 'c':3, 'x':100, 'y':200}
print('start:', d)
key = next(iter(d.keys()))
d.pop(key)
print('pop first item:', d)

start: {'a': 1, 'b': 2, 'c': 3, 'x': 100, 'y': 200}
pop first item: {'b': 2, 'c': 3, 'x': 100, 'y': 200}


If you're interested, here's Raymond Hettinger's pure Python "version" of his C dict implementation:
`http://code.activestate.com/recipes/578375/`